<h2>Team Members</h2>
<ul>
<li>Sohad Hossam Eldin 1190019</li>
<li>Bassant Hisham Mohamed 1190018</li>
<li>Yasmin Hashem Niazy</li>
<li>Mary Ashraf 1190322</li>
</ul>

<h4>Imports</h4>

In [41]:
import re
from pyarabic.araby import strip_tashkeel
from nltk import word_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sohad\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

<h4>Reading the dataset</h4>

In [16]:
dataset_txt = open(r"train.txt", "r").read()
print(type(dataset_txt))

<class 'str'>


<h4>Cleaning the dataset</h4>

In [17]:
# do we need to remove [ :ص]
chars_to_remove = r"\(\s*[a-zA-Z0-9_-]+\s*/\s*[a-zA-Z0-9_-]+\s*\)|[a-zA-Z0-9_-]+" 
dataset_cleaned = re.sub(chars_to_remove, "", dataset_txt)
# print(dataset_cleaned)

<h4>Segmenting dataset into sentences</h4>

In [39]:
# r'(\([^)]*\))' should we remove the brackets or not
dataset_sentences = re.split(r"\.|\n|،|:|[()]",dataset_cleaned)
print(type(dataset_sentences))
print(dataset_sentences[0])

<class 'list'>
قَوْلُهُ 


<h4>Removing Tashkeel</h4>

In [19]:
# dataset_without_tashkeel = strip_tashkeel(dataset_cleaned)
# print(dataset_without_tashkeel)

<h4>Tokenizing the sentence</h4>

In [50]:
tokenized_sentences = []
for sentence in dataset_sentences:
    tokens = word_tokenize(sentence, language="arabic", preserve_line=True)
    tokens.insert(0, "<s>")
    tokens.append("</s>")
    tokenized_sentences.append(tokens)

In [56]:
print(type(tokenized_sentences))

<class 'list'>
